# PyDrive2 Implementation

In this short notebook I implement the PyDrive2 library to download files that are storaged in a Drive Folder. This notebook only requieres internet connection and the installing of the listed libraries:

In [1]:
# ! pip install pydrive2 

To start using the library, it is necessary to follow the next steps: 


Set Up Google API Credentials:

1. Go to the Google Cloud Console.
2. Create a new project and enable the Google Drive API.
3. Create OAuth 2.0 credentials and download the credentials.json file.
4. Save the file as credentials.json in your working directory

After getting your JSON credentials, rename the file as _client_secrets.json_ and saved in the save folder as the code. To start using the drive library we need to pass an authentication section: 

In [2]:
# Libraries
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import os

In [3]:
# Step 1: Authentication
def authentication(cred: str = "mycreds.txt"):
    '''
    Do the authentication using credencials and returns the drive objetc

    Args: 
        cred (str) - Path to save the credentials

    Return: Drive object 
    '''
    try:
        gauth = GoogleAuth()
        gauth.LoadCredentialsFile(cred)  # Load previously saved credentials
        if not gauth.credentials:
            gauth.LoadClientConfigFile()  # Use the downloaded credentials.json
            gauth.LocalWebserverAuth()  # Authenticate using a local web server
            gauth.SaveCredentialsFile(cred)  # Save the credentials for future use
        return GoogleDrive(gauth)
    except Exception as e:
        print(e)

In [4]:
drive = authentication()

Any Drive Path is like: 

$https://drive.google.com/drive/folders/<folder-id>?usp=sharing$

We're only interested in the folder's id. So we implement the next code to extract that information: 

In [5]:
def get_file_id_from_link(link: str):
    '''
    Returns the object id of a folder or document 
    '''
    if "id=" in link:
        return link.split("id=")[1]
    elif "d/" in link:
        return link.split("d/")[1].split("/")[0]
    elif "folders/" in link:
        return link.split("folders/")[1].split("?")[0]
    else:
        raise ValueError("Invalid Google Drive link")

With the folders id, we implement the next function to get all the files in the folder: 

In [6]:
def download_folder_files(folder_id, download_path: str = './DriveFiles'):
    '''
    Download all the files in a Drive folder:

    Args:
        folder_id (str): The id in the link path 
        download_pat (str): This path will create a folder to save the files

    Return: True if all the documents were downloaded
    '''
    os.makedirs(download_path, exist_ok = True)
    
    # Query for files in the folder
    query = f"'{folder_id}' in parents and trashed=false"
    file_list = drive.ListFile({'q': query}).GetList()
    
    # Download each file
    try:    
        for file in file_list:
            print(f"Downloading {file['title']}...")
            file.GetContentFile( f"{download_path}/{file['title']}"   )
        print('True')
    except Exception as e:
        print('False: ', e)


Declare the link to your folder:

In [7]:
link = 'https://drive.google.com/drive/folders/1vqB3hahgJDEYzJg6jxQ4eXij2GNVMH34?usp=sharing'

In [8]:
id_file = get_file_id_from_link(link)
id_file

'1vqB3hahgJDEYzJg6jxQ4eXij2GNVMH34'

In [9]:
download_folder_files(id_file)

True


## Download just one document:

In [10]:
def download_just_one(file_id: str):
    file = drive.CreateFile({'id': file_id})
    file_name = file['title']  # Get the file name
    data = file.GetContentFile(file_name)  # Download the file
    print(f"File '{file_name}' downloaded successfully!")